# Bayesian Survival Analysis

This notebook aims to test a Bayesian Survival Analysis for non-small cell lung cancer (NSCLC) patients. It is an attempt to 
replicate, or perform a similar analysis, to that in [Jochems _et al_, International Journal of Radiation Oncology, **99** 
(2017)](https://doi.org/10.1016/j.ijrobp.2017.04.021) using a MCMC methodology like that in the [PyMC3 Bayesian Survival 
Analysis example](https://docs.pymc.io/notebooks/survival_analysis.html). The data from Jochems _et al_ is openly available 
[here](https://www.cancerdata.org/publication/developing-and-validating-survival-prediction-model-nsclc-patients-through-distributed).

In [1]:
%matplotlib inline

import pandas as pd
import requests
import os

First, download the raw data csv file from https://www.cancerdata.org/system/files/publications/Jochems-2017-MaastroDataUnbinned.csv.

In [2]:
csvfile = 'Jochems-2017-MaastroDataUnbinned.csv'
dataurl = 'https://www.cancerdata.org/system/files/publications/{}'

# check if file already exists
if not os.path.isfile(csvfile):
    # download the data
    data = requests.get(dataurl.format(csvfile))

    # output to a file
    fp = open(csvfile, 'w')
    fp.write(data.content.decode())
    fp.close()

# read in the data using pandas
table = pd.read_csv(csvfile)

In [3]:
table.head()

,yearrt,med,maxeso,gender,intake_who,age,chemo,ott,chemo3g,gtv1,...,CumultativeTotalTumorDose,meanlungdose,lungv20,CumOTT,OverallBaselineDysp,OverallPostRTDyspFullScore,DyspGT2,DeltaDyspGe1,TreatmentType,TwoYearSurvival
0,2010,17.0346,44.8989,0,2,67,1,21,2,,...,45.00,15.4178,33.8485,21,0,0,0,0,,0
1,2014,,,0,1,69,,36,,,...,67.00,,,36,1,1,0,0,2,0
2,2010,,,1,1,82,,24,,,...,52.25,,,24,0,1,0,1,,0
3,2013,17.2298,48.9217,1,1,77,1,36,2,,...,69.00,25.0428,11.9888,36,1,1,0,0,2,0
4,2013,,,1,2,83,,28,,,...,72.00,,,28,2,2,0,0,2,0


In [6]:
table.age.mean()

68.28980322003578

# Notes

Here are a few notes on where I should head with this analysis. I need to build a Bayesian model (to start with based on Figure 1 B of the above paper, which I should probably display in this notebook, e.g., with [daft](http://daft-pgm.org/)). This requires setting up conditional probability tables between the variables. What I would like to do is use a set of training data to find the posterior probability distributions on each of the required conditional probabilities in the tables, i.e., saying I don't know what the conditional probabilities are and I want to infer them based on a set of data. An example of setting up a Bayesian model (for the [simple rain/sprinkler/is grass wet? example](https://en.wikipedia.org/wiki/Bayesian_network#Example)) in PyMC3 is shown [here](https://gist.github.com/cs224/9a19b4ba2c7511e317be90c32a4d40d7#file-pymc3_rain_sprinkler_grass_simple_bayesian_network_with_evidence-py) (see the Stackoverflow post that discusses this [here](https://stackoverflow.com/questions/42470592/simple-bayesian-network-via-monte-carlo-markov-chain-ported-to-pymc3)). [Bernoulli distributions](https://docs.pymc.io/api/distributions/discrete.html#pymc3.distributions.discrete.Bernoulli) look like the correct way to go for the conditional probabilities in the hardcoded case, but, I think, as these would be contrained by multiple obersations a Binomial distributions would be correct (see, e.g., [this example](https://discourse.pymc.io/t/pymc3-differences-in-ways-observations-are-passed-to-model-difference-in-results/501)). However, in that example the conditional probabilities, i.e. the `0.2` in

```python
rain = pm.Bernoulli('rain', 0.2, shape=1, testval=tv)
```

or the `0.01` and `0.40` in

```python
sprinkler_p = pm.Deterministic('sprinkler_p', pm.math.switch(rain, 0.01, 0.40))
```

are hardcoded rather than fitted. So, I would required something like:

```python
prain = pm.Uniform('prain', 0., 1.)  # uniform prior on probability of rain (obviously "not rain" will be 1-prain)
rain = pm.Binomial('rain', prain, observed=raining)
```

In our case the probability distributions may be more complicated than uniform. I think to asign the truth tables the Deterministic class probably is required.

When wanting to do predictions on new data I can hopefully do something like what is shown in Section 3.5 of [this blog post](https://juanitorduz.github.io/intro_pymc3.html). 